# Tidy (Evictions) Data

## Why tidy data?

Tabular data comes in many shapes. Some are harder to work with than others due to how values and variables are organized in the form of rows and columns. 

An important part of the data journalist's job involves transforming data into a structure suitable for analysis. Here's a bit of background on the importance of "tidy" data from [Hadley Wickham](http://hadley.nz/), author of the [Tidy Data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) standard and creator of many good things in the realm of data analysis:

> The principles of tidy data provide a standard way to organise data values within a dataset. A standard makes initial data cleaning easier because you don’t need to start from scratch and reinvent the wheel every time. The tidy data standard has been designed to facilitate initial exploration and analysis of the data, and to simplify the development of data analysis tools that work well together.

That all sounds well and good, but a bit theoretical. Let's tidy up some real-world data in pursuit of a story.


## Meet the data

This tutorial demonstrates how to tidy up one example of "messy" data: [eviction notices in San Francisco](https://data.sfgov.org/Housing-and-Buildings/Eviction-Notices/5cei-gny5). The data covers more than 20 years of eviction notices. The goal is to reshape this data into a form that lets us explore some basic journalistic questions:

* What are the top reasons overall for evictions in San Francisco?
* What patterns or trends can we discern for the top reasons over time?

If we eyeball this data, we notice that it is "wide." There are more than 29 columns, the majority of which represent individual causes for eviction. Some important details to note:

* Each row reflects a single eviction notice.
* A property owner can cite one or more reasons for seeking to evict a tenant.
* Each reason column has a value of `TRUE` if the reason was cited, or `FALSE` if it was not.


![evictions-data](img/evictions_data_example.png)

## One flavor of messy

The evictions data exemplifies one of several major types of messy data: [when column headers are values instead of variable names](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html#column-headers-are-values-not-variable-names). What do we mean by this?

Here's a bit of background from Tidy Data:

> A common type of messy dataset is tabular data designed for presentation, where variables form both the rows and columns, and column headers are values, not variable names. While I would call this arrangement messy, in some cases it can be extremely useful. It provides efficient storage for completely crossed designs, and it can lead to extremely efficient computation if desired operations can be expressed as matrix operations.

The evictions data fits this description nicely. It's well suited -- even optimized -- for presentation and use in certain tools and types of analysis. But to answer our questions, the data is not in optimal form.


## Tidy evictions

The reasons for eviction are currently column headers. If we want to count reasons for eviction (for all time and by year), we'll need to transform them into values under a single, standardized column. We'll call this column `eviction_reason`.

Our end goal is to create one or more "unpivoted" data structures that enable us to more easily explore our questions. Here's an example of a tidy data structure that would make it easier to count and chart the reasons for eviction by year:

![tidied-up-data-example](img/tidied_up_data_example.png)

The above data structure is "long". It contains far fewer columns and many more rows than our original data set. Each row represents a unique combination of an `eviction_id`, `year`, cause for eviction, and whether it was cited. We'll ultimately filter this data to retain only the rows for cited reasons, to avoid over-counting.

But the critical first step is to transform the data into this form. Once we've tidied up the data, it will be much easier to filter, group and chart.

## Import the data

Let's import a locally stashed copy of the data and change the column names so that they're more user-friendly. Along the way, we'll convert the `File Date` field to a proper datetime, which will help downstream with data vetting and analysis.

In [1]:
import pandas as pd
import altair as alt

In [2]:
data = pd.read_csv('data/Eviction_Notices.csv', parse_dates=['File Date'], low_memory=False)
data.rename(columns={col: col.lower().replace(' ', '_') for col in data.columns}, inplace=True)

Take a gander at the columns. Notice the `file_date` is a datetime.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40178 entries, 0 to 40177
Data columns (total 29 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   eviction_id                          40178 non-null  object        
 1   address                              40178 non-null  object        
 2   city                                 40176 non-null  object        
 3   state                                40176 non-null  object        
 4   eviction_notice_source_zipcode       39495 non-null  object        
 5   file_date                            40178 non-null  datetime64[ns]
 6   non_payment                          40178 non-null  bool          
 7   breach                               40178 non-null  bool          
 8   nuisance                             40178 non-null  bool          
 9   illegal_use                          40178 non-null  bool          
 10  failure_to

## Slim down the data

Let's drop some of the fields that we don't plan to use for this particular analysis. 

> Note, there is likely a cleaner or more idiomatic way to do this, but we'll just go brute force for now.

In [4]:
to_drop = [
    'constraints_date',
    'address',
    'city',
    'state', 
    'eviction_notice_source_zipcode',
    'constraints_date',
    'supervisor_district',
    'neighborhoods_-_analysis_boundaries',
    'location'
]
subset = data.drop(to_drop, axis=1)

## Add a year column

Let's add a new `year` column, which will help us downstream when we count and chart the number of evictions by reason and year.

In [5]:
subset['year'] = subset.apply(lambda row: row.file_date.year, axis=1)

## Remove partial year

Let's filter out the 2019 data, since it's a partial year.

In [6]:
subset = subset[subset.file_date.dt.year < 2019]

In [7]:
subset.file_date.describe()

count                   40084
unique                   5442
top       2012-08-16 00:00:00
freq                      240
first     1997-01-02 00:00:00
last      2018-12-31 00:00:00
Name: file_date, dtype: object

### Pro Tip: Filtering by date

For the purposes of our analysis, we're focusing on the new `year` column, which we're using to filter out the partial year of data for 2019 and then for purposes of computing yearly counts of eviction causes. 

It's worth noting, however, that you can also filter the data set by the original `file_date` column, which is a `datetime` object. Below demonstrates how to filter the data for a particular date range.

In [8]:
from datetime import datetime
# March 4 to 8 2017
start = datetime(2017, 3, 4)
end = datetime(2017, 3, 8)

In [9]:
some_span_of_time = subset[(subset.file_date >= start) & (subset.file_date <= end)]

In [10]:
some_span_of_time.file_date.describe()
some_span_of_time.head(3)

,eviction_id,file_date,non_payment,breach,nuisance,illegal_use,failure_to_sign_renewal,access_denial,unapproved_subtenant,owner_move_in,...,substantial_rehab,ellis_act_withdrawal,condo_conversion,roommate_same_unit,other_cause,late_payments,lead_remediation,development,good_samaritan_ends,year
264,M170475,2017-03-06,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2017
427,M170522,2017-03-07,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,2017
789,M170470,2017-03-06,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2017


## Reshape the data

We're now ready to "melt" or unpivot our data. This will reshape the data from a wide to long format. Each row in the new data set will reflect a unique combination of the:

* eviction id
* year of the filing date
* reason for eviction
* reason "status"

This will include reasons that were not cited as a cause for eviction, as well as those that were cited.
We'll need to remove reasons that were not cited in a subsequent step to avoid over-counting.

In [11]:
melted = subset.melt(id_vars=['eviction_id','year'], var_name='eviction_reason', value_name='status')

## Filter reasons for eviction

The `melted` DataFrame includes rows for each "unpivoted" column, whether the reason was cited or not. To get an accurate count of the reasons cited, we'll filter for all records where the `status` is `True`.

In [12]:
filtered = melted[melted.status == True]
filtered.head()

,eviction_id,year,eviction_reason,status
40118,M172567,2017,non_payment,True
40140,M172648,2017,non_payment,True
40144,M172566,2017,non_payment,True
40160,M172650,2017,non_payment,True
40182,M172555,2017,non_payment,True


## Top Reasons

With our data unpivoted and filtered, let's move on to exploring our original questions. 

> We're gliding over much data vetting work at the moment. In a real-world project, much more time should be spent checking for missing/malformed values, duplicate records etc.

Our first question involved counting the most cited reasons for eviction notices over all years in the data set. Let's perform a basic `groupby` operation and sort the result from highest to lowest.

We should see that the top 3 reasons include "owner_move_in", "breach" (of contract, presumably), and "nuisance."

In [13]:
# Select the eviction reason and ID columns
# Peform a groupby that counts the IDs
# Sort from highest to lowest
filtered[['eviction_reason','eviction_id']].groupby(
    'eviction_reason', as_index=False
).count().sort_values('eviction_id' , ascending=False).head(10)

,eviction_reason,eviction_id
15,owner_move_in,10034
1,breach,8950
13,nuisance,6635
6,ellis_act_withdrawal,3859
12,non_payment,2387
10,late_payments,1634
2,capital_improvement,1539
14,other_cause,1364
16,roommate_same_unit,1343
4,demolition,1156


## Top Reasons Over Time

We've identified the top reasons for evictions for the full history of data. This is a good aggregate view of the data. But have these always been the leading reasons for eviction? Let's chart the top 3 to see if we find any patterns worthy of further analysis and reporting.

Our first step will involve counting the number of times each reason was cited for a given year. It's quite possible (and likely) that certain reasons will not have a count in every year, especially if we widen the analysis to include less-cited reasons such as "lead_remediation". 

Our analysis and visualization should reveal any missing years, so let's move forward.

In [14]:
target_vars = ['eviction_reason', 'year']
grouped = filtered.groupby(by=target_vars, as_index=False).eviction_id.count().sort_values(target_vars)
grouped.head()

,eviction_reason,year,eviction_id
0,access_denial,1997,13
1,access_denial,1998,12
2,access_denial,1999,11
3,access_denial,2000,13
4,access_denial,2001,9


### Filter

Let's filter the grouped data for the top 3 reasons.

In [15]:
top3 = grouped[grouped.eviction_reason.isin(['owner_move_in', 'breach', 'nuisance'])]

### Chart

Lastly, we'll chart the data with a few minor customizations.

We'll use a few altair features to clean up our chart and make it more digestible.

In [16]:
alt.Chart(top3).mark_line().encode(
    x=alt.X('year:O',  title='Year', ),
    y=alt.Y('eviction_id', title='Number of times cited'),
    color='eviction_reason'
).configure_axis(grid=True)

alt.Chart(...)

## Tidy is good, but not good enough

The trend lines above show that "owner move-ins" plummeted since the late 1990s, whereas "breach" and "nuisance" have increased in fits and starts, peaking around 2014 and leveling off in 2018.

Tidy data helped explore our original questions, but are these findings reliable? In particular, is the steep decline in "owner move-ins" a real pattern or a data anomaly? Perhaps the city database allowed just one reason for eviction to be flagged in the early years, and harried government employees left the "owner move-in" option checked by default, clicked `Enter`, and moved on to more important matters...

Before publishing conclusions from this analysis, we should consult agency sources for context. We should ask experts to verify our methodology and findings. We should seek out government reports, news stories, and academic research that have tread similar ground. We should find someone -- perhaps a teammate or industry colleague -- to replicate or review our analysis.

Data journalism is an iterative process. It often requires multiple rounds of technical work and reporting. Questioning our process and findings along the way is the surest path to "getting it right."